In [1]:
from tkinter import *
from PIL import Image,ImageTk
import cv2
import numpy as np

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from keras.models import load_model
model = load_model('main_model.h5')
cv2.ocl.setUseOpenCL(False)
emoji_dict = {0: "emojis/angry.png",1: "emojis/disguested.png",2: "emojis/fearful.png",3: "emojis/happy.png",4: "emojis/neutral.png",5: "emojis/sad.png",6: "emojis/surprised.png"}
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cascade_classifier = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
global cap
global last_frame
last_frame = np.zeros((300,300,3))
def show_vid():
    global cap
    cap = cv2.VideoCapture(0)

    ret, frame = cap.read()
    frame = cv2.resize(frame,(300,300))
    gray_img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #gray_img = img_gray.reshape(img_gray.shape[0],img_gray.shape[1],1)
    if not ret:
        print('Not opened')
    else:
        face = cascade_classifier.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)
        for (x,y,w,h) in face:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),5)
            roi = gray_img[y:y+h,x:x+w]
            roi_resized = cv2.resize(roi,(48,48))
            img_pred=np.expand_dims(np.expand_dims(cv2.resize(roi_resized, (48, 48)), -1), 0)
            #img_pred = np.expand_dims(roi_resized,axis=0)
            predict = model.predict_classes(img_pred)
            #cv2.putText(frame,text=emotion_dict[predict[0]],org=(x+20,y-60),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=2,color=(255,255,255),thickness=5,lineType=cv2.LINE_AA)
            show_text[0]=predict[0]
        #cv2.imshow('Facial Expression',cv2.resize(frame,(1280,720)))
        if ret is None:
            print("error")
        elif ret:
            global last_frame
            last_frame = frame.copy()
            pic_img = cv2.cvtColor(last_frame,cv2.COLOR_BGR2RGB)
            img = Image.fromarray(pic_img)
            img_tk = ImageTk.PhotoImage(image=img)
            photo_label.img_tk=img_tk
            photo_label.configure(image=img_tk)
            photo_label.after(15, show_vid)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            exit()        
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
show_text = [0]
def show_emoji():
    frame2=cv2.imread(emoji_dict[show_text[0]])
    frame2 = cv2.resize(frame2,(300,300))
    img2=Image.fromarray(frame2)
    imgtk2=ImageTk.PhotoImage(image=img2)
    label_emoji.imgtk2=imgtk2
    label_text.configure(text=emotion_dict[show_text[0]],font=('arial',45,'bold'))
    label_emoji.configure(image=imgtk2)
    label_emoji.after(15, show_emoji)

In [ ]:
root = Tk()
root.geometry("800x680")
root.configure(bg="gray")
root.title("Facial expression recognition")
label = Label(root,text="Facial Expression Recognition",foreground="white",bg="gray",font="helvetica 22 bold",pady=20)
label.grid(padx=10)
photo_label = Label(root,bg="black")
photo_label.grid(row=2,column=0)
label_text = Label(root)
label_text.grid(row=1,column=1,pady=5)
label_emoji = Label(root,bg = "black")
label_emoji.grid(row=2,column=1,pady=5)
b = Button(root,text="Exit",command=root.destroy,font="Arial 16 bold")
b.grid(row=3,column=0,pady=10,padx=10)
show_vid()
show_emoji()
root.mainloop()